# 달러1달리치

* 달러리치 앱 - 달러투자 가이드
  - https://m.blog.naver.com/PostView.naver?blogId=boot&logNo=222430430028&proxyReferer=

* 야후 finance 모듈 설치

In [1]:
!pip install yfinance --upgrade --no-cache-dir

* 야후 finance 모듈 import

In [2]:
import yfinance as yf

* get_ticker_hist(symbol, period) : period 기간의 symbol에 대해 시고저종 데이터프레임을 얻는다.
  - symbol 
    - KRW=X : 원/달러
    - DX-Y.NYB : 달러인덱스
  - period : 1mo(1달), 1y(1년)

In [3]:
def get_ticker_hist(symbol, period):
    df = yf.Ticker(symbol)
    hist = df.history(period=period, auto_adjust=False)
    return hist

def get_ticker_values(hist):
    low = hist['Low'].min()
    high = hist['High'].max()
    mid = (high + low) / 2.0
    close = hist['Close'].iloc[-1]
    values_dic = {
      "low": low,      
      "high": high,      
      "mid" : mid,
      "close" : close,
      "is_buy" : close < mid
    }
    return values_dic

## 1. 원달러 환율

In [4]:
# 원달러 환율 1년 데이터프레임 얻기
#hist = get_ticker_hist("KRW=X", "1mo")
hist = get_ticker_hist("KRW=X", "1y")

In [5]:
hist

,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
Date,,,,,,,,
2020-09-14,1185.780029,1186.880005,1179.630005,1186.969971,1186.969971,0,0,0
2020-09-15,1181.199951,1183.119995,1176.319946,1180.839966,1180.839966,0,0,0
2020-09-16,1179.849976,1181.170044,1169.760010,1179.250000,1179.250000,0,0,0
2020-09-17,1173.800049,1177.229980,1169.780029,1172.069946,1172.069946,0,0,0
2020-09-18,1168.760010,1168.760010,1158.640015,1168.650024,1168.650024,0,0,0
...,...,...,...,...,...,...,...,...
2021-09-07,1157.010010,1165.280029,1155.160034,1155.540039,1155.540039,0,0,0
2021-09-08,1161.130005,1167.739990,1161.130005,1161.000000,1161.000000,0,0,0
2021-09-09,1166.489990,1171.449951,1166.489990,1166.489990,1166.489990,0,0,0


* low : 기간(1년, 1달) 최저가
* high : 기간(1년, 1달) 최고가
* mid : low와 high의 중간가
* close : 현재가
* is_buy : 현재가가 기간(1년, 1달) 중간가보다 작으면 True, 아니면 False

In [6]:
usdkrw = get_ticker_values(hist)
usdkrw

{'close': 1173.2099609375,
 'high': 1186.8800048828125,
 'is_buy': False,
 'low': 1079.219970703125,
 'mid': 1133.0499877929688}

## 2. 달러 인덱스 (달러 지수)

In [8]:
# 달러인덱스 1년 데이터프레임 얻기
#hist = get_ticker_hist("DX-Y.NYB", "1mo")
hist = get_ticker_hist("DX-Y.NYB", "1y")
hist

,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
Date,,,,,,,,
2020-09-14,93.290001,93.330002,92.870003,93.070000,93.070000,0,0,0
2020-09-15,93.059998,93.139999,92.790001,93.050003,93.050003,0,0,0
2020-09-16,93.080002,93.279999,92.779999,93.209999,93.209999,0,0,0
2020-09-17,93.150002,93.589996,92.870003,92.970001,92.970001,0,0,0
2020-09-18,92.860001,93.010002,92.769997,92.930000,92.930000,0,0,0
...,...,...,...,...,...,...,...,...
2021-09-07,92.199997,92.570000,92.099998,92.519997,92.519997,0,0,0
2021-09-08,92.540001,92.860001,92.470001,92.650002,92.650002,0,0,0
2021-09-09,92.709999,92.760002,92.379997,92.480003,92.480003,0,0,0


* low : 기간(1년, 1달) 최저가
* high : 기간(1년, 1달) 최고가
* mid : low와 high의 중간가
* close : 현재가
* is_buy : 현재가가 기간(1년, 1달) 중간가보다 작으면 True, 아니면 False

In [9]:
dxy = get_ticker_values(hist)
dxy

{'close': 92.6449966430664,
 'high': 94.73999786376953,
 'is_buy': False,
 'low': 89.20999908447266,
 'mid': 91.9749984741211}

## 3. 달러 갭 비율
* 산출 공식 : 달러 지수 / (원/달러 환율) * 100 = 달러 갭 비율
* mid : 달러 지수 중간값 / (원/달러 환율 중간값) * 100
* close : 달러 지수 현재가 / (원/달러 환율 현재가) * 100
* is_buy : 현재가가 기간(1년, 1달) 중간값보다 크면 True, 아니면 False

In [12]:
def get_gap(usdkrw, dxy):
    gap_dic = {}
    mid = dxy.get("mid", 0) / usdkrw.get("mid", 0) * 100.0
    close = dxy.get("close", 0) / usdkrw.get("close", 0) * 100.0
    if dxy.get("mid", 0) and usdkrw.get("mid", 0):
        gap_dic["mid"] = mid
    if dxy.get("close", 0) and usdkrw.get("close", 0):
        gap_dic["close"] = close
    gap_dic["is_buy"] = close > mid
    return gap_dic

In [13]:
gap_rate = get_gap(usdkrw, dxy)
gap_rate

{'close': 7.896710710590519, 'is_buy': False, 'mid': 8.11747049689098}

## 4. 적정 환율
* 산출 공식 : 현재의 달러 지수 / 52주 중간 달러 갭 비율 * 100 = 적정 환율

In [14]:
def get_proper_usdkrw(dxy, gap_rate):
    value = 0
    if dxy.get("close", 0) and gap_rate.get("mid", 0):
        value = dxy.get("close", 0) / gap_rate.get("mid", 0) * 100.0
    return value

In [15]:
get_proper_usdkrw(dxy, gap_rate)

1141.3037679477834